### <center>Method of variable directions

In [1]:
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt

In [2]:
def zero_t_func(x_values, y_values):
    x, y = sp.symbols('x y')
    expr = x**2 + y**2
    func = sp.lambdify((x, y), expr, "numpy")
    X, Y = np.meshgrid(x_values, y_values, indexing='ij')
    return func(X, Y)


def true_func(t_values, x_values, y_values):
    x, y, t = sp.symbols('x y t')
    expr = t + x**2 + y**2
    func = sp.lambdify((t, x, y), expr, "numpy")
    T, X, Y = np.meshgrid(t_values, x_values, y_values, indexing='ij')
    return func(T, X, Y)


def boundary_x_func(t_values, y_values, value):
    x, y, t = sp.symbols('x y t')
    expr = t + x**2 + y**2
    expr = expr.subs(x, value)
    func = sp.lambdify((t, y), expr, "numpy")
    T, Y = np.meshgrid(t_values, y_values, indexing='ij')
    return func(T, Y)


def boundary_y_func(t_values, x_values, value):
    x, y, t = sp.symbols('x y t')
    expr = t + x**2 + y**2
    expr = expr.subs(y, value)
    func = sp.lambdify((t, x), expr, "numpy")
    T, X = np.meshgrid(t_values, x_values, indexing='ij')
    return func(T, X)


def f_func():
    '''
    x, y, t = sp.symbols('x y t')
    expr = t + x**2 + y**2
    result = sp.diff(expr, t, 1) - sp.diff(expr, x, 2) - sp.diff(expr, y, 2)
    return int(result) if bool(result.free_symbols) == False else result
    '''
    return -3


def mu(bound_k, bound_k1, tau):
    return (bound_k1 + bound_k) / 2 - tau * (bound_k - bound_k1) / 4


def boundary_x_extra_func(u, t_values, y_values, value_left, value_right):
    for k in range(2, len(t)-1, 2):
        u[k, :, 0] = mu(true_func(t_values[k], 0, y_values).flatten(), true_func(t_values[k+1], 0, y_values).flatten(), tau)
        u[k, :, -1] = mu(true_func(t_values[k], 1, y_values).flatten(), true_func(t_values[k+1], 1, y_values).flatten(), tau)
    return u

def boundary_y_extra_func(u, t_values, x_values, value_down, value_up):
    for k in range(2, len(t)-1, 2):
        u[k, 0, :] = mu(true_func(t_values[k], x_values, 0).flatten(), true_func(t_values[k+1], x_values, 0).flatten(), tau)
        u[k, -1, :] = mu(true_func(t_values[k], x_values, 1).flatten(), true_func(t_values[k+1], x_values, 1).flatten(), tau)
    return u


def tridiagona_matrix_algorithm(A, b):
    n = A.shape[0]
    c = np.zeros(n-1) # subdiagonal elements
    d = np.zeros(n) # main diagonal
    e = np.zeros_like(c) # supradiagonal elements
    
    for i in range(n - 1):
        c[i] = A[i+1, i]
        d[i] = A[i, i]
        e[i] = A[i, i+1]
    d[n-1] = A[n-1, n-1]
    
    alpha = np.zeros(n-1)
    beta = np.zeros_like(alpha)

    # Straight direction
    alpha[0] = -e[0] / d[0]
    beta[0] = b[0] / d[0]
    
    for i in range(1, n - 1):
        alpha[i] = -e[i] / (c[i - 1] * alpha[i - 1] + d[i])
        beta[i] = (b[i] - c[i - 1] * beta[i - 1]) / (c[i - 1] * alpha[i - 1] + d[i])

    # Reverse direction
    x = np.zeros_like(b)
    x[n - 1] = (b[n - 1] - c[n - 2] * beta[n - 2]) / (c[n - 2] * alpha[n - 2] + d[n - 1])
    
    for i in range(n - 2, -1, -1):
        x[i] = alpha[i] * x[i + 1] + beta[i]

    return x


def alternating_direction_implicit(u, n_t, n_x, n_y, h_x, h_y, tau):
    for k in range(0, 2 * n_t - 1, 2):
        for i in range(1, n_x):
            A = np.zeros((n_y + 1, n_y + 1))
            b = np.zeros(n_y + 1)
            A[0, 0] = A[-1, -1] = 1
            b[0] = u[k, i, 0]
            b[-1] = u[k, i, -1]

            for j in range(1, n_y):
                b[j] = (u[k, i-1, j] - 2 * u[k, i, j] + u[k, i+1, j]) / h_x**2 + 2 * u[k, i, j] / tau + f_func()
                A[j, j-1] = A[j, j+1] = -1 / h_y**2
                A[j, j] = 2 * (1 / h_y**2 + 1 / tau)

            u[k+1, i, 1:-1] = tridiagona_matrix_algorithm(A, b)[1:-1]

        for j in range(1, n_y):
            A = np.zeros((n_x + 1, n_x + 1))
            b = np.zeros(n_x + 1)
            A[0, 0] = A[-1, -1] = 1
            b[0] = u[k+1, 0, j]
            b[-1] = u[k+1, -1, j]

            for i in range(1, n_x):
                b[i] = (u[k+1, i, j-1] - 2 * u[k+1, i, j] + u[k+1, i, j+1]) / h_x**2 + 2 * u[k+1, i, j] / tau + f_func()
                A[i, i-1] = A[i, i+1] = -1 / h_x**2
                A[i, i] = 2 * (1 / h_x**2 + 1 / tau)

            u[k+2, 1:-1, j] = tridiagona_matrix_algorithm(A, b)[1:-1]

    return u

In [3]:
n_x = n_y = 10
n_t = 100
left_x, right_x, left_y, right_y, left_t, right_t = 0, 1, 0, 1, 0, 1
x = np.linspace(left_x, right_x, n_x+1)
y = np.linspace(left_y, right_y, n_y+1)
t = np.linspace(left_t, right_t, 2*n_t+1)

h_x = np.diff(x)[0]
h_y = np.diff(y)[0]
tau = np.diff(t)[0]

u = np.zeros((2*n_t+1, n_x+1, n_y+1))

# Boundary conditions
u[:, :, 0] = boundary_y_func(t, x, 0)
u[:, :, -1] = boundary_y_func(t, x, 1)
u = boundary_y_extra_func(u, t, x, 0, 1)

u[:, 0, :] = boundary_x_func(t, y, 0)
u[:, -1, :] = boundary_x_func(t, y, 1)
u = boundary_x_extra_func(u, t, y, 0, 1)

# Initial conditions
u[0, :, :] = zero_t_func(x, y)

In [4]:
u = alternating_direction_implicit(u, n_t, n_x, n_y, h_x, h_y, tau)

In [5]:
print(f"Максимальная погрешность: {(u - true_func(t, x, y)).max()}")

Максимальная погрешность: 0.0025062500000005983
